Unduh data korpus Wikipedia Bahasa Indonesia melalui link berikut https://dumps.wikimedia.org/idwiki/latest/. Cari file dengan nama “idwiki-latest-pages-articles.xml.bz2”. Data korpus setiap waktunya mengalami peningkatan. Sekitar agustus 2019 terdapat 406855 kosa kata dengan ukuran 512 MB.

Data korpus tersebut masih menggunakan format XML sehingga perlu di olah ke dalam bentuk teks. library Gensim sudah menyediakan teknik pra poses ini. Untuk mengkonversi format tersebut dapat menggunakan kode berikut:

In [1]:
from __future__ import print_function
 
# Ignore warnings dari gensim
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
 
import logging
import os.path
import sys
 
from gensim.corpora import WikiCorpus
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "idwiki-latest-pages-articles.xml.bz2"
namaFileOutput = "wiki.id.text"
 
space = " "
i = 0
 
# Write file ke variabel namaFileOutput encoder utf-8
output = open(namaFileOutput, 'w', encoding='utf-8')
 
# lower=False: huruf kecil dan besar dibedakan
wiki = WikiCorpus(namaFileInput, lemmatize=None, dictionary={}, lower=False)
for text in wiki.get_texts():
    output.write(' '.join(text) + '\n')
    i = i + 1
    if i % 10000 == 0:
        logger.info("Saved " + str(i) + " articles")
 
output.close()
logger.info("Finished Saved " + str(i) + " articles")

2023-11-02 19:25:14,683: INFO: running D:\User\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py -f C:\Users\hp\AppData\Roaming\jupyter\runtime\kernel-bf5a0c2b-76eb-4157-aca3-1e2cd76e0d2e.json
2023-11-02 19:25:47,883: INFO: Saved 10000 articles
2023-11-02 19:26:12,520: INFO: Saved 20000 articles
2023-11-02 19:26:31,238: INFO: Saved 30000 articles
2023-11-02 19:26:47,384: INFO: Saved 40000 articles
2023-11-02 19:27:05,189: INFO: Saved 50000 articles
2023-11-02 19:27:21,657: INFO: Saved 60000 articles
2023-11-02 19:27:37,704: INFO: Saved 70000 articles
2023-11-02 19:27:54,780: INFO: Saved 80000 articles
2023-11-02 19:28:20,790: INFO: Saved 90000 articles
2023-11-02 19:28:38,791: INFO: Saved 100000 articles
2023-11-02 19:28:55,235: INFO: Saved 110000 articles
2023-11-02 19:29:10,242: INFO: Saved 120000 articles
2023-11-02 19:29:32,519: INFO: Saved 130000 articles
2023-11-02 19:29:49,375: INFO: Saved 140000 articles
2023-11-02 19:30:00,772: INFO: Saved 150000 articles
2023-11-02 19:30:0

Dari kode di atas akan menghasilkan output file dari rangkaian beberapa artikel, di mana satu barisnya mewakili satu artikel.

Setelah data berhasil terkonversi, selanjutnya dilakukan training Word2Vec menggunakan kode berikut:

In [2]:
import multiprocessing
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
 
program = os.path.basename(sys.argv[0])
logger = logging.getLogger(program)
 
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info("running %s" % ' '.join(sys.argv))
 
namaFileInput = "wiki.id.text"
namaFileOutput = "w2vec_wiki_id300_0.txt"
 
 
# size 300 = 300 dimensi vektor, window 10 = 10 pengaruh kata disekitarnya, min_count 5 = kata-kata yang muncul < 5 kali akan dikeluarkan dari kosakata dan diabaikan selama pelatihan, sg 0 = cbow / sg 1 = skip gram 
model = Word2Vec(LineSentence(namaFileInput), vector_size=300, window=10, min_count=5, sg=0, workers=multiprocessing.cpu_count())
 
# trim unneeded model memory = use (much) less RAM
model.init_sims(replace=True)
model.wv.save_word2vec_format(namaFileOutput, binary=False)

2023-11-02 19:47:13,087: INFO: running D:\User\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py -f C:\Users\hp\AppData\Roaming\jupyter\runtime\kernel-bf5a0c2b-76eb-4157-aca3-1e2cd76e0d2e.json
2023-11-02 19:47:13,087: INFO: collecting all words and their counts
2023-11-02 19:47:13,106: INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-11-02 19:47:15,788: INFO: PROGRESS: at sentence #10000, processed 10738098 words, keeping 353874 word types
2023-11-02 19:47:17,415: INFO: PROGRESS: at sentence #20000, processed 17334952 words, keeping 490368 word types
2023-11-02 19:47:18,661: INFO: PROGRESS: at sentence #30000, processed 22148080 words, keeping 591133 word types
2023-11-02 19:47:19,786: INFO: PROGRESS: at sentence #40000, processed 26549341 words, keeping 676278 word types
2023-11-02 19:47:21,045: INFO: PROGRESS: at sentence #50000, processed 31270076 words, keeping 775169 word types
2023-11-02 19:47:22,134: INFO: PROGRESS: at sentence #60000, processed 35

2023-11-02 19:48:04,310: INFO: EPOCH 0 - PROGRESS: at 0.08% examples, 717893 words/s, in_qsize 8, out_qsize 1
2023-11-02 19:48:05,327: INFO: EPOCH 0 - PROGRESS: at 0.22% examples, 776160 words/s, in_qsize 5, out_qsize 0
2023-11-02 19:48:06,338: INFO: EPOCH 0 - PROGRESS: at 0.32% examples, 793661 words/s, in_qsize 15, out_qsize 0
2023-11-02 19:48:07,343: INFO: EPOCH 0 - PROGRESS: at 0.47% examples, 806682 words/s, in_qsize 7, out_qsize 0
2023-11-02 19:48:08,346: INFO: EPOCH 0 - PROGRESS: at 0.70% examples, 806041 words/s, in_qsize 15, out_qsize 0
2023-11-02 19:48:09,351: INFO: EPOCH 0 - PROGRESS: at 0.84% examples, 810610 words/s, in_qsize 13, out_qsize 0
2023-11-02 19:48:10,345: INFO: EPOCH 0 - PROGRESS: at 0.99% examples, 804745 words/s, in_qsize 8, out_qsize 0
2023-11-02 19:48:11,370: INFO: EPOCH 0 - PROGRESS: at 1.21% examples, 809953 words/s, in_qsize 12, out_qsize 0
2023-11-02 19:48:12,384: INFO: EPOCH 0 - PROGRESS: at 1.40% examples, 810478 words/s, in_qsize 5, out_qsize 0
2023-1

2023-11-02 19:49:18,934: INFO: EPOCH 0 - PROGRESS: at 29.89% examples, 787655 words/s, in_qsize 1, out_qsize 0
2023-11-02 19:49:19,934: INFO: EPOCH 0 - PROGRESS: at 33.60% examples, 792261 words/s, in_qsize 8, out_qsize 0
2023-11-02 19:49:20,955: INFO: EPOCH 0 - PROGRESS: at 37.41% examples, 796825 words/s, in_qsize 12, out_qsize 0
2023-11-02 19:49:21,962: INFO: EPOCH 0 - PROGRESS: at 41.42% examples, 801330 words/s, in_qsize 13, out_qsize 0
2023-11-02 19:49:22,962: INFO: EPOCH 0 - PROGRESS: at 45.44% examples, 805047 words/s, in_qsize 15, out_qsize 0
2023-11-02 19:49:23,969: INFO: EPOCH 0 - PROGRESS: at 50.06% examples, 809150 words/s, in_qsize 13, out_qsize 0
2023-11-02 19:49:24,978: INFO: EPOCH 0 - PROGRESS: at 50.76% examples, 809889 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:49:25,984: INFO: EPOCH 0 - PROGRESS: at 51.34% examples, 810431 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:49:26,985: INFO: EPOCH 0 - PROGRESS: at 51.86% examples, 810651 words/s, in_qsize 0, out_qsize

2023-11-02 19:50:33,426: INFO: EPOCH 0 - PROGRESS: at 93.36% examples, 826371 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:34,425: INFO: EPOCH 0 - PROGRESS: at 93.87% examples, 826623 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:35,433: INFO: EPOCH 0 - PROGRESS: at 94.34% examples, 826860 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:36,446: INFO: EPOCH 0 - PROGRESS: at 94.82% examples, 827108 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:37,457: INFO: EPOCH 0 - PROGRESS: at 95.39% examples, 827166 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:38,473: INFO: EPOCH 0 - PROGRESS: at 95.99% examples, 827386 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:39,470: INFO: EPOCH 0 - PROGRESS: at 96.66% examples, 827554 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:50:40,485: INFO: EPOCH 0 - PROGRESS: at 97.32% examples, 827654 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:50:41,485: INFO: EPOCH 0 - PROGRESS: at 97.94% examples, 827802 words/s, in_qsize 0, out_qsize 0
2

2023-11-02 19:51:47,938: INFO: EPOCH 1 - PROGRESS: at 26.73% examples, 863221 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:51:48,954: INFO: EPOCH 1 - PROGRESS: at 27.26% examples, 863478 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:51:49,963: INFO: EPOCH 1 - PROGRESS: at 27.75% examples, 863414 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:51:50,971: INFO: EPOCH 1 - PROGRESS: at 28.22% examples, 863202 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:51:51,972: INFO: EPOCH 1 - PROGRESS: at 28.70% examples, 862540 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:51:52,993: INFO: EPOCH 1 - PROGRESS: at 29.35% examples, 863255 words/s, in_qsize 4, out_qsize 0
2023-11-02 19:51:53,995: INFO: EPOCH 1 - PROGRESS: at 31.98% examples, 866361 words/s, in_qsize 3, out_qsize 0
2023-11-02 19:51:55,000: INFO: EPOCH 1 - PROGRESS: at 35.91% examples, 870705 words/s, in_qsize 1, out_qsize 0
2023-11-02 19:51:56,006: INFO: EPOCH 1 - PROGRESS: at 39.65% examples, 874366 words/s, in_qsize 11, out_qsize 1


...

2023-11-02 19:55:31,115: INFO: EPOCH 2 - PROGRESS: at 86.87% examples, 870114 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:32,131: INFO: EPOCH 2 - PROGRESS: at 87.71% examples, 870143 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:33,142: INFO: EPOCH 2 - PROGRESS: at 88.43% examples, 870110 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:34,142: INFO: EPOCH 2 - PROGRESS: at 89.14% examples, 870064 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:35,139: INFO: EPOCH 2 - PROGRESS: at 89.86% examples, 869872 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:36,154: INFO: EPOCH 2 - PROGRESS: at 90.60% examples, 869835 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:37,158: INFO: EPOCH 2 - PROGRESS: at 91.30% examples, 869812 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:55:38,160: INFO: EPOCH 2 - PROGRESS: at 91.97% examples, 869631 words/s, in_qsize 2, out_qsize 0
2023-11-02 19:55:39,166: INFO: EPOCH 2 - PROGRESS: at 92.45% examples, 869573 words/s, in_qsize 0, out_qsize 0
2

2023-11-02 19:56:45,231: INFO: EPOCH 3 - PROGRESS: at 21.15% examples, 865696 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:56:46,238: INFO: EPOCH 3 - PROGRESS: at 21.72% examples, 865905 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:56:47,265: INFO: EPOCH 3 - PROGRESS: at 22.19% examples, 864936 words/s, in_qsize 12, out_qsize 0
2023-11-02 19:56:48,265: INFO: EPOCH 3 - PROGRESS: at 22.69% examples, 864049 words/s, in_qsize 5, out_qsize 0
2023-11-02 19:56:49,272: INFO: EPOCH 3 - PROGRESS: at 23.34% examples, 864205 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:56:50,267: INFO: EPOCH 3 - PROGRESS: at 23.96% examples, 864024 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:56:51,274: INFO: EPOCH 3 - PROGRESS: at 24.50% examples, 864055 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:56:52,282: INFO: EPOCH 3 - PROGRESS: at 25.02% examples, 864437 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:56:53,288: INFO: EPOCH 3 - PROGRESS: at 25.49% examples, 864626 words/s, in_qsize 0, out_qsize 0


2023-11-02 19:57:59,841: INFO: EPOCH 3 - PROGRESS: at 80.72% examples, 845766 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:00,835: INFO: EPOCH 3 - PROGRESS: at 81.35% examples, 845642 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:01,851: INFO: EPOCH 3 - PROGRESS: at 82.13% examples, 845611 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:02,868: INFO: EPOCH 3 - PROGRESS: at 82.84% examples, 845690 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:03,869: INFO: EPOCH 3 - PROGRESS: at 83.52% examples, 845932 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:04,886: INFO: EPOCH 3 - PROGRESS: at 84.23% examples, 846070 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:05,879: INFO: EPOCH 3 - PROGRESS: at 84.96% examples, 846178 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:58:06,896: INFO: EPOCH 3 - PROGRESS: at 85.69% examples, 846281 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:58:07,885: INFO: EPOCH 3 - PROGRESS: at 86.48% examples, 846459 words/s, in_qsize 0, out_qsize 0
2

2023-11-02 19:59:13,553: INFO: EPOCH 4 - PROGRESS: at 15.86% examples, 860541 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:59:14,570: INFO: EPOCH 4 - PROGRESS: at 16.40% examples, 860877 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:59:15,575: INFO: EPOCH 4 - PROGRESS: at 16.86% examples, 860582 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:59:16,572: INFO: EPOCH 4 - PROGRESS: at 17.44% examples, 860612 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:59:17,581: INFO: EPOCH 4 - PROGRESS: at 18.02% examples, 859204 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:59:18,580: INFO: EPOCH 4 - PROGRESS: at 18.64% examples, 858291 words/s, in_qsize 0, out_qsize 1
2023-11-02 19:59:19,585: INFO: EPOCH 4 - PROGRESS: at 19.29% examples, 858572 words/s, in_qsize 1, out_qsize 0
2023-11-02 19:59:20,593: INFO: EPOCH 4 - PROGRESS: at 19.86% examples, 858856 words/s, in_qsize 0, out_qsize 0
2023-11-02 19:59:21,593: INFO: EPOCH 4 - PROGRESS: at 20.36% examples, 858853 words/s, in_qsize 0, out_qsize 0
2

2023-11-02 20:00:28,093: INFO: EPOCH 4 - PROGRESS: at 76.76% examples, 867657 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:29,088: INFO: EPOCH 4 - PROGRESS: at 77.44% examples, 867638 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:30,103: INFO: EPOCH 4 - PROGRESS: at 78.08% examples, 867592 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:31,113: INFO: EPOCH 4 - PROGRESS: at 78.76% examples, 867445 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:32,110: INFO: EPOCH 4 - PROGRESS: at 79.40% examples, 867484 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:33,118: INFO: EPOCH 4 - PROGRESS: at 80.02% examples, 867288 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:34,126: INFO: EPOCH 4 - PROGRESS: at 80.72% examples, 867359 words/s, in_qsize 1, out_qsize 0
2023-11-02 20:00:35,143: INFO: EPOCH 4 - PROGRESS: at 81.37% examples, 867249 words/s, in_qsize 0, out_qsize 0
2023-11-02 20:00:36,148: INFO: EPOCH 4 - PROGRESS: at 82.16% examples, 867107 words/s, in_qsize 0, out_qsize 0
2

Terdapat beberapa parameter diantaranya:

<b>vector_size</b>: Ukuran dimensi vector yang mewakili setiap token atau kata. Jika memiliki data yang terbatas, nilai size yang digunakan sebaiknya lebih kecil karena akan mempengaruhi kata unik di sekitarnya. Namun, jika memiliki banyak data maka dapat bereksperimen dengan berbagai ukuran.

<b>window</b>: Jarak maksimum antara kata target dengan kata di sekitarnya. Jika nilai window yang digunakan besar, maka terdapat banyak kata terkait disekitarnya (baik di posisi kiri dan kanan dari kata konteks). Secara teori, jika window lebih kecil akan memberikan istilah yang lebih spesifik terhadap suatu konteks kata.

<b>min_count</b>: Frekuensi minimal jumlah kata. Model akan mengabaikan kata-kata yang tidak memenuhi nilai min_count. Kata yang jarang muncul biasanya tidak terlalu penting, jadi lebih baik untuk dihilangkan. Parameter ini mungkin lebih berpengaruh pada effisiensi penggunaan memory dan ukuran file model.

<b>sg</b>: 0 untuk arsitektur Word2Vec CBOW dan 1 untuk arsitektur Word2Vec Skip-gram.

<b>workers</b>: Berapa banyak threads yang digunakan untuk melakukan multiprocessing.

Model yang sudah disimpan dapat dilakukan skenario penggunaannya sebagaimana pada kode di bawah ini untuk melihat kedekatan hubungan antara kata.

In [7]:
from gensim.models import Word2Vec

# Melatih model Word2Vec
# Ganti 'sentences' dengan data pelatihan Anda (list of lists of words)
model = Word2Vec(sentences, vector_size=300, window=10, min_count=5, sg=0, workers=4)

# Menyimpan model ke dalam file
model.save("nama_file_model")


NameError: name 'sentences' is not defined

In [6]:
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModel
import torch

# Memuat model Word2Vec
namaFileModel = "w2vec_wiki_id300_0.txt"
model_w2v = Word2Vec.load(namaFileModel)

# Memuat model BERT
bert_model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModel.from_pretrained(bert_model_name)

# Fungsi untuk mendapatkan vektor kata dari model BERT
def get_bert_word_vector(word):
    inputs = tokenizer(word, return_tensors="pt")
    with torch.no_grad():
        outputs = bert_model(**inputs)
    word_vector = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)
    return word_vector

# Uji kedekatan kata-kata menggunakan model Gensim
sim_gensim = model_w2v.wv.most_similar("Jakarta")
print("10 kata terdekat dari Jakarta (Gensim): {}".format(sim_gensim))

# Uji kedekatan kata-kata menggunakan model BERT
sim_bert = model_w2v.wv.similar_by_vector(get_bert_word_vector("Jakarta"), topn=10, restrict_vocab=None)
print("10 kata terdekat dari Jakarta (BERT): {}".format(sim_bert))


2023-11-02 20:56:11,085: INFO: loading Word2Vec object from w2vec_wiki_id300_0.txt


UnpicklingError: invalid load key, '5'.